- Create Lakebase Database Instance 
- Create a synced table

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance, SyncedTableSpec, SyncedTableSchedulingPolicy, SyncedDatabaseTable

In [0]:
# Initialize the workspace client 
w = WorkspaceClient()

In [0]:
# create the database instance 
# host = 'https://adb-984752964297111.11.azuredatabricks.net' # Databricks workspace URL
lakebase_instance_name = 'zg-mfg-lakebase-demo' # name of the database instance
cu = 'CU_1' # size of the database instance, currently can be CU_1, CU_2, CU_4, or CU_8
node_count = 2 # number of database nodes, need at least 2 for failover

database_instance = DatabaseInstance(
  name=lakebase_instance_name,
  capacity=cu,
  node_count=node_count
)

w.database.create_database_instance(
  database_instance = database_instance
)

In [0]:
# wait for the database instance to become available before creating the synced table
w.database.wait_get_database_instance_database_available(name=lakebase_instance_name)

In [0]:
# create the synced table
destination_table_name = "zg.mfg_lakebase_demo.recommended_routes_synced_table"
source_table_full_name = "zg.mfg_lakebase_demo.recommended_routes"
primary_key_columns = ["part_id"]

table_spec = SyncedTableSpec(
    source_table_full_name=source_table_full_name,
    primary_key_columns=primary_key_columns,
    scheduling_policy=SyncedTableSchedulingPolicy.TRIGGERED, 
)

synced_table = SyncedDatabaseTable(
  name=destination_table_name,
  database_instance_name=lakebase_instance_name,
  logical_database_name='pg_mfg_db',
  spec=table_spec
)

w.database.create_synced_database_table(synced_table=synced_table)